# Build you Own HTTP Server

## 1. Bind to a Port

In this stage, you'll create a TCP server that listens on port 4221.

[TCP](https://www.cloudflare.com/en-ca/learning/ddos/glossary/tcp-ip/) is the underlying protocol used by HTTP servers.

To Test, run `telnet 127.0.0.1 4221` in different terminal while the main() function is executed.

Notes
- To learn how HTTP works, you'll implement your server from scratch using TCP primitives instead of using Python's built-in HTTP libraries.



In [1]:
import socket

def main():

    server_socket = socket.create_server(("localhost", 4221), reuse_port=True)

    server_socket.accept() # wait for client

if __name__ == "__main__":

    main()

## 2. Respond with 200


An HTTP response consists of three parts, each separated by a carriage return and line feed (`\r\n`):

1. **Status line**
2. **Zero or more headers**, each ending with a CRLF
3. **Optional response body**

**Minimal Response Format**

In this stage, your server's response will only contain a **status line**. Here's the exact response your server must send:

```

HTTP/1.1 200 OK\r\n\r\n

```

**Breakdown of the Response**

```

// Status line
HTTP/1.1  // HTTP version
200       // Status code
OK        // Optional reason phrase
\r\n      // CRLF marking end of status line

// Headers (empty)
\r\n      // CRLF marking end of headers

// Response body (empty)

```

**Reference**

See the [MDN Web Docs on HTTP responses](https://developer.mozilla.org/en-US/docs/Web/HTTP/Messages#http_responses) or the [HTTP/1.1 specification](https://datatracker.ietf.org/doc/html/rfc7230#section-3) for more information.

**Tests**

The tester will run your server with:

```

\$ ./your\_program.sh

```

Then send an HTTP GET request:

```

\$ curl -v http://localhost:4221

```

Your server must respond with:

```

HTTP/1.1 200 OK\r\n\r\n

```

**Notes**

- You can ignore the contents of the request. Request parsing will be handled in later stages.
- This challenge uses HTTP/1.1.
- You will build the server using TCP primitives instead of built-in HTTP libraries.



In [ ]:
import socket  

def main():

    def parse_request_target(request_data: bytes):
        request_line = request_data.decode().split("\r\n")
        request_target = request_line[0].split(" ")[1]
        return request_target

    def handle_request(request_target: str):
        return b"HTTP/1.1 200 OK\r\n\r\n"

    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        # Return a new socket representing the connection, and the address of the client.
        client_socket, address_info = server_socket.accept() # wait for client
        hostaddr= address_info[0]
        port = address_info[1]

        request_data = client_socket.recv(1024) # bytes

        request_target =  parse_request_target(request_data)

        response =  handle_request(request_target)

        client_socket.send(response)

    except Exception as e:
        print(f"An error occured: {e}")

    finally:
        client_socket.close()

if __name__ == "__main__":
    main()


Listening to http://localhost:4221


When sending the `curl -v http://localhost:4221`, it will response
```bash
*   Trying 127.0.0.1:4221...
* Connected to localhost (127.0.0.1) port 4221 (#0)
> GET / HTTP/1.1
> Host: localhost:4221
> User-Agent: curl/7.88.1
> Accept: */*
> 
< HTTP/1.1 200 OK
* no chunk, no close, no size. Assume close to signal end
< 
* Closing connection 0
```

`>` lines are request lines — what `curl` sends to the server.

`<` lines are response lines — what `curl` receives from the server.

`*` lines are diagnostic output from `curl`, like connection info or behavior notes.

## 3. Extract URL path

In this stage, your server will extract the URL path from an HTTP request and respond with either a 200 or 404 status code depending on the path.

**Structure of an HTTP Request**

An HTTP request consists of three parts, each separated by a carriage return and line feed (`\r\n`):

1. **Request line**
2. **Zero or more headers**, each ending with a CRLF
3. **Optional request body**

**Example HTTP Request**

```

GET /index.html HTTP/1.1\r\n
Host: localhost:4221\r\n
User-Agent: curl/7.64.1\r\n
Accept: */*\r\n
\r\n

```

**Breakdown of the Request**

```

// Request line
GET                          // HTTP method
/index.html                  // Request target (URL path)
HTTP/1.1                     // HTTP version
\r\n                         // CRLF marking end of request line

// Headers
Host: localhost:4221\r\n     // Specifies the server's host and port
User-Agent: curl/7.64.1\r\n  // Describes the client's user agent
Accept: */*\r\n              // Specifies acceptable media types
\r\n                         // CRLF marking end of headers

// Request body (empty)

```

The "request target" specifies the **URL path**. In this example, it is `/index.html`.

**Tests**

The tester will run your server with:

```

\$ ./your\_program.sh

```

Then it will send two HTTP GET requests:

1. A request with a random path:

```

\$ curl -v http://localhost:4221/abcdefg

```

Expected response:

```

HTTP/1.1 404 Not Found\r\n\r\n

```

2. A request to the root path:

```

\$ curl -v http://localhost:4221

```

Expected response:

```

HTTP/1.1 200 OK\r\n\r\n

```

**Notes**

- You can ignore the headers for now. Header parsing will be covered in a later stage.
- The request target in this stage uses the **origin form** (i.e., a URL path), which is the most common form. Other forms exist for proxying and special scenarios.
- For more details, refer to the [MDN Web Docs on HTTP requests](https://developer.mozilla.org/en-US/docs/Web/HTTP/Messages#http_requests) or the [HTTP/1.1 specification](https://datatracker.ietf.org/doc/html/rfc7230#section-3.1.1).

In [ ]:
import socket  

def main():

    def parse_request_target(request_data: bytes):
        request_line = request_data.decode().split("\r\n")
        request_target = request_line[0].split(" ")[1]
        return request_target

    def handle_request(request_target: str):
        if request_target == '/': # Check for root path
            return b"HTTP/1.1 200 OK\r\n\r\n"
        else: # Random path
            return b"HTTP/1.1 404 Not Found\r\n\r\n"

    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        # Return a new socket representing the connection, and the address of the client.
        client_socket, address_info = server_socket.accept() # wait for client
        hostaddr= address_info[0]
        port = address_info[1]

        request_data = client_socket.recv(1024) # bytes

        request_target =  parse_request_target(request_data)

        response =  handle_request(request_target)

        client_socket.send(response)

    except Exception as e:
        print(f"An error occured: {e}")

    finally:
        client_socket.close()

if __name__ == "__main__":
    main()


## 4. Respond with body

In this stage, you will implement the `/echo/{str}` endpoint, which accepts a string and returns it in the response body.

**Response Body**

A response body is used to return content to the client. It can contain anything that can be represented in bytes, such as a string, HTML page, or file content.

Your `/echo/{str}` endpoint must return a **200 OK** response with:

- The response body set to the given string
- A `Content-Type` header
- A `Content-Length` header

**Example Request**

```

GET /echo/abc HTTP/1.1\r\n
Host: localhost:4221\r\n
User-Agent: curl/7.64.1\r\n
Accept: */*\r\n
\r\n

```

**Expected Response**

```

HTTP/1.1 200 OK\r\n
Content-Type: text/plain\r\n
Content-Length: 3\r\n
\r\n
abc

```

**Breakdown of the Response**

```

// Status line
HTTP/1.1 200 OK
\r\n                          // CRLF marking end of status line

// Headers
Content-Type: text/plain\r\n  // Format of the response body
Content-Length: 3\r\n         // Size of the response body in bytes
\r\n                          // CRLF marking end of headers

// Response body
abc                           // Echoed string from the request

```

**Tests**

The tester will run your server using:

```

\$ ./your\_program.sh

```

Then send a GET request:

```

\$ curl -v http://localhost:4221/echo/abc

```

Expected response:

```

HTTP/1.1 200 OK\r\n
Content-Type: text/plain\r\n
Content-Length: 3\r\n
\r\n
abc

```

**Notes**

- Both `Content-Type` and `Content-Length` headers are required so the client can parse the body properly.
- Each header ends with `\r\n`, and the header section ends with another `\r\n`.
- For more details, see the [MDN Web Docs on HTTP responses](https://developer.mozilla.org/en-US/docs/Web/HTTP/Messages#http_responses) or the [HTTP/1.1 specification](https://datatracker.ietf.org/doc/html/rfc7230).


In [2]:
# curl --verbose 127.0.0.1:4221/echo/abc

In [ ]:
import socket  

def main():

    def parse_client_request(request_data: bytes):
        """Parse the request from the client."""
        request_line = request_data.decode().split("\r\n")

        http_method = request_line[0].split(" ")[0]
        request_target = request_line[0].split(" ")[1]
        http_version = request_line[0].split(" ")[2]

        # server_host = request_line[1].split(" ")[1]

        # client_user_agent = request_line[2].split(" ")[1]

        # acceptable_media_types = request_line[3].split(" ")[1]

        return request_target
    
    def route_request(request_target: str) -> tuple[str, str]:
        """Returns (status, body)"""
        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            body = ""        

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            body = parts[2]

        else:
            status = "404 Not Found"
            body = ""

        return (status, body)

    def create_response(route_result: tuple):
        status = route_result[0]
        body = route_result[1]
        headers = [
            f"HTTP/1.1 {status}",
            "Content-Type: text/plain",
            f"Content-Length: {len(body)}",
            ""
        ]            
        response = "\r\n".join(headers) + "\r\n" + body
        print(response.encode()  )          
        return response.encode()   
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        client_socket, address_info = server_socket.accept() 
        hostaddr = address_info[0]
        port = address_info[1]

        request_data = client_socket.recv(1024)

        request_target = parse_client_request(request_data)

        route_result = route_request(request_target)
        server_response  = create_response(route_result)
        print(server_response)
        client_socket.send(server_response)

    except Exception as e:
        print(f"An error occured: {e}")

    finally:
        client_socket.close()

if __name__ == "__main__":
    main()


Listening to http://localhost:4221
b'HTTP/1.1 200 OK\r\nContent-Type: text/plain\r\nContent-Length: 9\r\n\r\npineapple'
b'HTTP/1.1 200 OK\r\nContent-Type: text/plain\r\nContent-Length: 9\r\n\r\npineapple'


## 5. Read header

In this stage, you will implement the `/user-agent` endpoint, which reads the `User-Agent` request header and returns its value in the response body.

**The User-Agent Header**

The `User-Agent` header describes the client's user agent (e.g., browser, curl, etc.).

Your server must:

- Extract the value of the `User-Agent` header
- Return a **200 OK** response
- Include the `User-Agent` value in the response body
- Set appropriate `Content-Type` and `Content-Length` headers

**Example Request**

```

// Request line
GET /user-agent HTTP/1.1\r\n

// Headers
Host: localhost:4221\r\n
User-Agent: foobar/1.2.3\r\n
Accept: */*\r\n
\r\n

// Request body (empty)

```

**Expected Response**

```

HTTP/1.1 200 OK\r\n
Content-Type: text/plain\r\n
Content-Length: 12\r\n
\r\n
foobar/1.2.3

```

**Breakdown of the Response**

```

// Status line
HTTP/1.1 200 OK
\r\n                          // End of status line

// Headers
Content-Type: text/plain\r\n  // Response body format
Content-Length: 12\r\n        // Length of the User-Agent string
\r\n                          // End of headers

// Response body
foobar/1.2.3                  // Echoed User-Agent value

```

**Tests**

The tester will run your program with:

```

\$ ./your\_program.sh

```

Then send the following request:

```

\$ curl -v --header "User-Agent: foobar/1.2.3" http://localhost:4221/user-agent

```

Expected response:

```

HTTP/1.1 200 OK\r\n
Content-Type: text/plain\r\n
Content-Length: 12\r\n
\r\n
foobar/1.2.3

```

**Notes**

- Header names are case-insensitive.


In [26]:
        # """Parse the request from the client."""
        # request_line = request_data.decode().split("\r\n")
        # #print(request_line)
        # # ['GET /user-agent HTTP/1.1', 'Host: 127.0.0.1:4221', 'User-Agent: curl/7.88.1', 'Accept: */*', '', '']
        # http_method = request_line[0].split(" ")[0]
        # request_target = request_line[0].split(" ")[1]
        # http_version = request_line[0].split(" ")[2]

        # # server_host = request_line[1].split(" ")[1]

        # client_user_agent = request_line[2].split(" ")[1]

        # # acceptable_media_types = request_line[3].split(" ")[1]

        # return request_target, client_user_agent 

In [59]:
import socket  

def main():

    def parse_client_request(request_data: bytes):
        """Parse the request from the client."""
        request_parts = request_data.decode().split("\r\n")

        try:
            # Request line
            request_line = request_parts[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = request_parts[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")

            return request_target, client_user_agent 
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_parts}")
            print(f"len: {len(request_parts)}")
        
    
    def route_request(request_target: str, client_user_agent: str) -> tuple[str, str]:
        """Returns (status, body)"""
        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            body = client_user_agent

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            body = parts[2]

        else:
            status = "404 Not Found"
            body = ""

        return (status, body)

    def create_response(route_result: tuple):
        status = route_result[0]
        body = route_result[1]
        headers = [
            f"HTTP/1.1 {status}",
            "Content-Type: text/plain",
            f"Content-Length: {len(body)}",
            ""
        ]            
        response = "\r\n".join(headers) + "\r\n" + body
        return response.encode()   
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        client_socket, address_info = server_socket.accept() 
        hostaddr = address_info[0]
        port = address_info[1]

        request_data = client_socket.recv(1024)

        request_target, client_user_agent = parse_client_request(request_data)

        route_result = route_request(request_target, client_user_agent )
        server_response  = create_response(route_result)
        client_socket.send(server_response)

    except Exception as e:
        print(f"An error occured: {e}")

    finally:
        client_socket.close()

if __name__ == "__main__":
    main()


Listening to http://localhost:4221


## 6. Concurrent connections

In this stage, you will add support for handling **concurrent TCP connections** to your server.

**Tests**

The tester will execute your server with:

```

$ ./your_program.sh

```

Then it will initiate multiple concurrent TCP connections. Each connection will send a single HTTP GET request after a short delay:

```

$ (sleep 3 && printf "GET / HTTP/1.1\r\n\r\n") | nc localhost 4221 &
$ (sleep 3 && printf "GET / HTTP/1.1\r\n\r\n") | nc localhost 4221 &
$ (sleep 3 && printf "GET / HTTP/1.1\r\n\r\n") | nc localhost 4221 &

```

Your server must respond to **each** request with:

```

HTTP/1.1 200 OK\r\n\r\n

```

This means your server must not block while handling a single request. It should handle all connections either in parallel (e.g., using threads or processes) or asynchronously.



In [ ]:
import socket  
import threading

def main():

    def parse_client_request(request_data: bytes):
        """Parse the request from the client."""
        request_parts = request_data.decode().split("\r\n")

        try:
            # Request line
            request_line = request_parts[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = request_parts[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")

            return request_target, client_user_agent 
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_parts}")
            print(f"len: {len(request_parts)}")
        
    
    def route_request(request_target: str, client_user_agent: str) -> tuple[str, str]:
        """Returns (status, body)"""
        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            body = client_user_agent

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            body = parts[2]

        else:
            status = "404 Not Found"
            body = ""

        return (status, body)
    
    def handle_request(client_socket):
        try:
            request_data = client_socket.recv(1024)

            request_target, client_user_agent = parse_client_request(request_data)

            route_result = route_request(request_target, client_user_agent )
            server_response  = create_response(route_result)
            client_socket.send(server_response)

        except Exception as e:
            print(f"An error occured: {e}")

        finally:
            client_socket.close()

    def create_response(route_result: tuple):
        status = route_result[0]
        body = route_result[1]
        headers = [
            f"HTTP/1.1 {status}",
            "Content-Type: text/plain",
            f"Content-Length: {len(body)}",
            ""
        ]            
        response = "\r\n".join(headers) + "\r\n" + body
        return response.encode() 
      
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        while True: # Keep the server in accept mode
            client_socket, address_info = server_socket.accept() 
            hostaddr = address_info[0]
            port = address_info[1]

            thread = threading.Thread(target=handle_request, args=(client_socket,))
            thread.start() 

    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    main()


Listening to http://localhost:4221


## 7. Return a file

In this stage, you will implement the `/files/{filename}` endpoint, which returns a requested file from the server's specified directory.

**Tests**

The tester will start your server with a `--directory` flag that points to the file storage location:

```bash
$ ./your_program.sh --directory /tmp/
````

Then, two `GET` requests will be sent to the `/files/{filename}` endpoint.

**First Request**

A file that exists in the directory:

```bash
$ echo -n 'Hello, World!' > /tmp/foo
$ curl -i http://localhost:4221/files/foo
```

Expected response:

```
HTTP/1.1 200 OK\r\n
Content-Type: application/octet-stream\r\n
Content-Length: 13\r\n
\r\n
Hello, World!
```

**Second Request**

A file that does **not** exist in the directory:

```bash
$ curl -i http://localhost:4221/files/non_existant_file
```

Expected response:

```
HTTP/1.1 404 Not Found\r\n\r\n
```

**Requirements for 200 OK Response**

* `Content-Type` must be `application/octet-stream`
* `Content-Length` must match the size of the file in bytes
* The response body must contain the full file content


In [ ]:
import os
import sys
import socket  
import threading

def main():

    def parse_client_request(request_data: bytes):
        """Parse the request from the client."""
        request_parts = request_data.decode().split("\r\n")

        try:
            # Request line
            request_line = request_parts[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = request_parts[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")

            return request_target, client_user_agent 
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_parts}")
            print(f"len: {len(request_parts)}")
        
    
    def route_request(request_target: str, client_user_agent: str) -> dict:
        """Returns (status, body)"""
        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            content_type = "text/plain"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            content_type = "text/plain"
            body = client_user_agent

        # Read content from the file (take full path from the argument CLI)
        elif parts[1] == "files":
            filename = parts[2]
            directory = sys.argv[2]
            full_path = os.path.join(directory, filename)
            print(sys.argv)
            print(f"Path to the file: {full_path}" )
            try:
                with open(full_path, mode="r") as f: 
                    # mode is read as string not bytes
                    file_bytes = f.read()
                    status = "200 OK"
                    content_type = "application/octet-stream"
                    body = file_bytes

            except FileNotFoundError:
                status = "404 Not Found"
                content_type = "text/plain"
                body = ""                
                

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            content_type = "text/plain"
            body = parts[2]

        else:
            status = "404 Not Found"
            content_type = ""
            body = ""

        return {"status": status,
                "content_type": content_type,
                "body": body}
    
    def handle_request(client_socket):
        try:
            request_data = client_socket.recv(1024)

            request_target, client_user_agent = parse_client_request(request_data)

            route_result = route_request(request_target, client_user_agent )
            server_response  = create_response(route_result)
            client_socket.send(server_response)

        except Exception as e:
            print(f"An error occured: {e}")

        finally:
            client_socket.close()

    def create_response(route_result: dict):
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]
        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Length: {len(body)}",
            ""
        ]            
        response = "\r\n".join(headers) + "\r\n" + body
        return response.encode() 
      
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        while True:
            client_socket, address_info = server_socket.accept() 
            hostaddr = address_info[0]
            port = address_info[1]

            thread = threading.Thread(target=handle_request, args=(client_socket,))
            thread.start() 

    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    main()


## 8. Read request body

In this stage, you will add support for the `POST` method to the `/files/{filename}` endpoint. This allows the client to send data in the request body, which your server must write to a file.

**Request Body**

A request body carries data from the client to the server. The `Content-Type` and `Content-Length` headers specify the format and size of this data.

**Example Request**

```

// Request line
POST /files/number HTTP/1.1\r\n

// Headers
Host: localhost:4221\r\n
User-Agent: curl/7.64.1\r\n
Accept: */*\r\n
Content-Type: application/octet-stream\r\n
Content-Length: 5\r\n
\r\n

// Request Body
12345

````

**Tests**

The tester will run your server with the `--directory` flag:

```bash
$ ./your_program.sh --directory /tmp/
````

Then, it will send a `POST` request with raw data:

```bash
$ curl -v --data "12345" -H "Content-Type: application/octet-stream" http://localhost:4221/files/file_123
```

Expected response:

```
HTTP/1.1 201 Created\r\n\r\n
```

**Expected Behavior**

* The server must create a new file in the directory specified by `--directory`
* The filename must match the value in the URL path
* The contents of the file must exactly match the request body



In [ ]:
import os
import sys
import socket  
import threading

def main():

    def parse_client_request(request_data: bytes, client_socket: socket) -> dict:
        """
        Parse the request from the client.
        ['GET /user-agent HTTP/1.1', 'Host: 127.0.0.1:4221', 'User-Agent: curl/7.88.1', 'Accept: */*', '', '']
        """
        headers_raw, body_start = request_data.decode().split("\r\n\r\n")
        header_lines = headers_raw.split("\r\n")

        try:
            # Request line
            request_line = header_lines[0]
            request_method = request_line.split(" ")[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = header_lines[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")

            if request_method == "POST":
                content_length = int(headers.get('content-length', ""))
                while len(body_start) < content_length:
                    body_start += client_socket.recv(content_length - len(body_start))

            return {"request_target": request_target,
                    "request_method": request_method,
                    "client_user_agent": client_user_agent,
                    "body": body_start}
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_data}")
            print(f"len headers: {len(headers_raw)}")
            print(f"len body: {content_length}")
            print(f"len received body: {len(body_start)}")
        
    
    def route_request(parsed_request: dict) -> dict:
        """Returns (status, body)"""
        request_target = parsed_request["request_target"]
        client_user_agent = parsed_request["client_user_agent"]
        request_method = parsed_request["request_method"]

        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            content_type = "text/plain"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            content_type = "text/plain"
            body = client_user_agent

        elif parts[1] == "files":
            filename = parts[2]
            directory = sys.argv[2]
            full_path = os.path.join(directory, filename)
            print(sys.argv)
            print(f"Path to the file: {full_path}" )
            print(request_method)
            try:
                if request_method == "GET":
                    with open(full_path, mode="rb") as f: 
                        file_bytes = f.read()
                        status = "200 OK"
                        content_type = "application/octet-stream"
                        body = file_bytes
                elif request_method == "POST":
                    body = parsed_request["body"]
                    with open(full_path, mode="w") as f: 
                        file_bytes = f.write(body)
                        status = "201 Created"
                        content_type = "application/octet-stream"
                else:
                    status = "405 Method Not Allowed"
                    content_type = "text/plain"
                    body = ""

            except FileNotFoundError:
                status = "404 Not Found"
                content_type = "text/plain"
                body = ""                
                

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            content_type = "text/plain"
            body = parts[2]

        else:
            status = "404 Not Found"
            content_type = ""
            body = ""

        return {"status": status,
                "content_type": content_type,
                "body": body}
    
    def create_response(route_result: dict):
        """
        Get the route result. Returns the response as bytes.
        """
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]
        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Length: {len(body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(body, str):
            body_bytes = body.encode()
        else:
            body_bytes = body

        return header_bytes + body_bytes
    
    def handle_request(client_socket):
        try:
            request_data = b""
            while b"\r\n\r\n" not in request_data:
                request_data += client_socket.recv(1024)

            parsed_request = parse_client_request(request_data, client_socket)

            route_result = route_request(parsed_request)
            server_response  = create_response(route_result)
            client_socket.send(server_response)

        except Exception as e:
            print(f"An error occured: {e}")

        finally:
            client_socket.close()
      
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        while True:
            client_socket, address_info = server_socket.accept() 
            hostaddr = address_info[0]
            port = address_info[1]

            thread = threading.Thread(target=handle_request, args=(client_socket,))
            thread.start() 

    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    main()


## Extension: HTTP Compression 

## 9. Compression Headers

Welcome to the HTTP Compression extension. In this extension, you'll add support for compression to your HTTP server.

In this stage, you'll add support for the `Accept-Encoding` and `Content-Encoding` headers.

**Accept-Encoding and Content-Encoding**

An HTTP client uses the `Accept-Encoding` header to specify the compression schemes it supports. In the following example, the client specifies that it supports the `gzip` compression scheme:

```

GET /echo/foo HTTP/1.1
Host: localhost:4221
User-Agent: curl/7.81.0
Accept: */*
Accept-Encoding: gzip

```

The server chooses one of the compression schemes listed in `Accept-Encoding` and compresses the response body with it. Then, the server sends a response with the compressed body and a `Content-Encoding` header. `Content-Encoding` specifies the compression scheme used.

Example response with a gzip-compressed body:

```

HTTP/1.1 200 OK
Content-Encoding: gzip
Content-Type: text/plain
Content-Length: 23

````

If the server does not support any of the compression schemes requested by the client, it will send a regular (uncompressed) response and omit the `Content-Encoding` header.

For this extension, assume that your server only supports the `gzip` compression scheme.

You do not need to compress the body in this stage. Compression will be implemented in a later stage.

**Tests**

The tester will execute your program like this:

```bash
$ ./your_program.sh
````

**First Request**

A request with the following header:

```bash
$ curl -v -H "Accept-Encoding: gzip" http://localhost:4221/echo/abc
```

Expected response:

```
HTTP/1.1 200 OK
Content-Type: text/plain
Content-Encoding: gzip
```

**Second Request**

A request with the following header:

```bash
$ curl -v -H "Accept-Encoding: invalid-encoding" http://localhost:4221/echo/abc
```

Expected response:

```
HTTP/1.1 200 OK
Content-Type: text/plain
```

**Notes**

Your server should only support `gzip` for now. Support for multiple compression schemes in `Accept-Encoding` will be added in a later stage. Another method for HTTP compression using the `TE` and `Transfer-Encoding` headers exists, but it is not covered in this extension.



In [ ]:
import os
import sys
import socket  
import threading

def main():

    def parse_client_request(request_data: bytes, client_socket: socket) -> dict:
        """
        Parse the request from the client.
        ['GET /echo/strawberry HTTP/1.1', 'Host: localhost:4221', 'Accept-Encoding: gzip']
        """
        headers_raw, body_start = request_data.decode().split("\r\n\r\n")
        header_lines = headers_raw.split("\r\n")

        try:
            # Request line
            request_line = header_lines[0]
            request_method = request_line.split(" ")[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = header_lines[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")
            accept_encoding = headers.get('accept-encoding', "")

            if request_method == "POST":
                content_length = int(headers.get('content-length', ""))
                while len(body_start) < content_length:
                    body_start += client_socket.recv(content_length - len(body_start))

            return {"request_target": request_target,
                    "request_method": request_method,
                    "client_user_agent": client_user_agent,
                    "accept_encoding": accept_encoding,
                    "body": body_start}
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_data}")
            print(f"len headers: {len(headers_raw)}")
            print(f"len body: {content_length}")
            print(f"len received body: {len(body_start)}")
        
    
    def route_request(parsed_request: dict) -> dict:
        """Returns (status, body)"""
        request_target = parsed_request["request_target"]
        client_user_agent = parsed_request["client_user_agent"]
        request_method = parsed_request["request_method"]

        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            content_type = "text/plain"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            content_type = "text/plain"
            body = client_user_agent

        elif parts[1] == "files":
            filename = parts[2]
            directory = sys.argv[2]
            full_path = os.path.join(directory, filename)
            print(sys.argv)
            print(f"Path to the file: {full_path}" )
            print(request_method)
            try:
                if request_method == "GET":
                    with open(full_path, mode="rb") as f: 
                        file_bytes = f.read()
                        status = "200 OK"
                        content_type = "application/octet-stream"
                        body = file_bytes
                elif request_method == "POST":
                    body = parsed_request["body"]
                    with open(full_path, mode="w") as f: 
                        file_bytes = f.write(body)
                        status = "201 Created"
                        content_type = "application/octet-stream"
                else:
                    status = "405 Method Not Allowed"
                    content_type = "text/plain"
                    body = ""

            except FileNotFoundError:
                status = "404 Not Found"
                content_type = "text/plain"
                body = ""                
                

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            content_type = "text/plain"
            body = parts[2]

        else:
            status = "404 Not Found"
            content_type = ""
            body = ""

        return {"status": status,
                "content_type": content_type,
                "body": body}
    
    def apply_compressed_response(parsed_request: dict, route_result: dict):
        """Apply the compression to the response if needed."""
        accept_encoding = parsed_request["accept_encoding"]
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]        

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Encoding: {accept_encoding}",
            f"Content-Length: {len(body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(body, str):
            body_bytes = body.encode()
        else:
            body_bytes = body

        return header_bytes + body_bytes
    
    def create_response(route_result: dict):
        """
        Get the route result. Returns the response as bytes.
        """
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Length: {len(body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(body, str):
            body_bytes = body.encode()
        else:
            body_bytes = body

        return header_bytes + body_bytes
    
    def handle_request(client_socket):
        try:
            request_data = b""
            while b"\r\n\r\n" not in request_data:
                request_data += client_socket.recv(1024)

            parsed_request = parse_client_request(request_data, client_socket)

            route_result = route_request(parsed_request)

            accept_encoding = parsed_request["accept_encoding"]

            if accept_encoding == "gzip":
                server_response = apply_compressed_response(parsed_request, route_result)
            else:
                server_response  = create_response(route_result)

            client_socket.send(server_response)

        except Exception as e:
            print(f"An error occured: {e}")

        finally:
            client_socket.close()
      
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        while True:
            client_socket, address_info = server_socket.accept() 
            hostaddr = address_info[0]
            port = address_info[1]

            thread = threading.Thread(target=handle_request, args=(client_socket,))
            thread.start() 

    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    main()


## 10. Multiple Compression Schemes

In this stage, you'll add support for `Accept-Encoding` headers that contain multiple compression schemes.

**Multiple Compression Schemes**

A client can specify multiple supported compression schemes using a comma-separated list:

```

Accept-Encoding: encoding-1, encoding-2, encoding-3

````

For this extension, your server should only support the `gzip` compression scheme.

You do not need to compress the response body in this stage. Compression will be handled in a later stage.

**Tests**

The tester will run your program using:

```bash
$ ./your_program.sh
````

**First Request**

A request where the `Accept-Encoding` header includes `gzip` along with some invalid encodings:

```bash
$ curl -v -H "Accept-Encoding: invalid-encoding-1, gzip, invalid-encoding-2" http://localhost:4221/echo/abc
```

Expected response:

```
HTTP/1.1 200 OK
Content-Type: text/plain
Content-Encoding: gzip
```

**Second Request**

A request where the `Accept-Encoding` header contains only invalid encodings:

```bash
$ curl -v -H "Accept-Encoding: invalid-encoding-1, invalid-encoding-2" http://localhost:4221/echo/abc
```

Expected response:

```
HTTP/1.1 200 OK
Content-Type: text/plain
```



In [ ]:
import os
import sys
import socket  
import threading

def main():

    def parse_client_request(request_data: bytes, client_socket: socket) -> dict:
        """
        Parse the request from the client.
        "GET /echo/pineapple HTTP/1.1\r\nHost: localhost:4221\r\nAccept-Encoding: encoding-1, gzip, encoding-2\r\n\r\n"
        """
        headers_raw, body_start = request_data.decode().split("\r\n\r\n")
        header_lines = headers_raw.split("\r\n")

        try:
            # Request line
            request_line = header_lines[0]
            request_method = request_line.split(" ")[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = header_lines[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")
            accept_encoding = headers.get('accept-encoding', "")

            # If received accept-encoding
            if accept_encoding:
                # List format
                accept_encoding = accept_encoding.split(", ")

            if request_method == "POST":
                content_length = int(headers.get('content-length', ""))
                while len(body_start) < content_length:
                    body_start += client_socket.recv(content_length - len(body_start))

            return {"request_target": request_target,
                    "request_method": request_method,
                    "client_user_agent": client_user_agent,
                    "accept_encoding": accept_encoding,
                    "body": body_start}
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_data}")
            print(f"len headers: {len(headers_raw)}")
            print(f"len body: {content_length}")
            print(f"len received body: {len(body_start)}")
        
    
    def route_request(parsed_request: dict) -> dict:
        """Returns (status, body)"""
        request_target = parsed_request["request_target"]
        client_user_agent = parsed_request["client_user_agent"]
        request_method = parsed_request["request_method"]

        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            content_type = "text/plain"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            content_type = "text/plain"
            body = client_user_agent

        elif parts[1] == "files":
            filename = parts[2]
            directory = sys.argv[2]
            full_path = os.path.join(directory, filename)
            try:
                if request_method == "GET":
                    with open(full_path, mode="rb") as f: 
                        file_bytes = f.read()
                        status = "200 OK"
                        content_type = "application/octet-stream"
                        body = file_bytes
                elif request_method == "POST":
                    body = parsed_request["body"]
                    with open(full_path, mode="w") as f: 
                        file_bytes = f.write(body)
                        status = "201 Created"
                        content_type = "application/octet-stream"
                else:
                    status = "405 Method Not Allowed"
                    content_type = "text/plain"
                    body = ""

            except FileNotFoundError:
                status = "404 Not Found"
                content_type = "text/plain"
                body = ""                
                

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            content_type = "text/plain"
            body = parts[2]

        else:
            status = "404 Not Found"
            content_type = ""
            body = ""

        return {"status": status,
                "content_type": content_type,
                "body": body}
    
    def apply_compressed_response(response_encoding: list, route_result: dict):
        """Apply the compression to the response if needed."""
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]    

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Encoding: {', '.join(response_encoding)}",
            f"Content-Length: {len(body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(body, str):
            body_bytes = body.encode()
        else:
            body_bytes = body

        return header_bytes + body_bytes
    
    def create_response(route_result: dict):
        """
        Get the route result. Returns the response as bytes.
        """
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Length: {len(body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(body, str):
            body_bytes = body.encode()
        else:
            body_bytes = body

        return header_bytes + body_bytes
    
    def handle_request(client_socket):
        try:
            request_data = b""
            while b"\r\n\r\n" not in request_data:
                request_data += client_socket.recv(1024)

            parsed_request = parse_client_request(request_data, client_socket)

            route_result = route_request(parsed_request)

            accept_encoding = parsed_request["accept_encoding"]

            server_accepted_encoding = ["gzip"]
            response_encoding = []

            for e in accept_encoding:
                if e in server_accepted_encoding:
                    response_encoding.append(e)
                    break

            if response_encoding:
                server_response = apply_compressed_response(response_encoding, route_result)
            else:
                server_response  = create_response(route_result)

            client_socket.send(server_response)

        except Exception as e:
            print(f"An error occured: {e}")

        finally:
            client_socket.close()
      
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        while True:
            client_socket, address_info = server_socket.accept() 
            hostaddr = address_info[0]
            port = address_info[1]

            thread = threading.Thread(target=handle_request, args=(client_socket,))
            thread.start() 

    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    main()


## 11. Gzip Compression

In this stage, you'll add support for gzip compression to your HTTP server.

**Tests**

The tester will execute your program using:

```bash
$ ./your_program.sh
````

Then it will send a `GET` request to the `/echo/{str}` endpoint with `Accept-Encoding: gzip`:

```bash
$ curl -v -H "Accept-Encoding: gzip" http://localhost:4221/echo/abc | hexdump -C
```

**Expected Response**

```
HTTP/1.1 200 OK
Content-Encoding: gzip
Content-Type: text/plain
Content-Length: 23
```

The response body must be the gzip-compressed version of the string `abc`, sent as binary data. Example of expected body in hexadecimal:

```
1f 8b 08 00 00 00 00 00 00 03 4b 4c 4a 06 00 c2
41 24 35 03 00 00 00
```

**Requirements**

* Return status code `200 OK`
* Set `Content-Type` to `text/plain`
* Set `Content-Encoding` to `gzip`
* Set `Content-Length` to the size of the compressed body
* Set the response body to the gzip-compressed version of the string

**Notes**

To verify the compressed body manually, run:

```bash
$ echo -n abc | gzip | hexdump -C
```

Short strings like `abc` may result in a larger compressed output. This is expected behavior.


In [ ]:
import os
import sys
import gzip
import socket  
import threading

def main():

    def parse_client_request(request_data: bytes, client_socket: socket) -> dict:
        """
        Parse the request from the client.
        "GET /echo/pineapple HTTP/1.1\r\nHost: localhost:4221\r\nAccept-Encoding: encoding-1, gzip, encoding-2\r\n\r\n"
        """
        headers_raw, body_start = request_data.decode().split("\r\n\r\n")
        header_lines = headers_raw.split("\r\n")

        try:
            # Request line
            request_line = header_lines[0]
            request_method = request_line.split(" ")[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = header_lines[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")
            accept_encoding = headers.get('accept-encoding', "")

            # If received accept-encoding
            if accept_encoding:
                # List format
                accept_encoding = accept_encoding.split(", ")

            if request_method == "POST":
                content_length = int(headers.get('content-length', ""))
                while len(body_start) < content_length:
                    body_start += client_socket.recv(content_length - len(body_start))

            return {"request_target": request_target,
                    "request_method": request_method,
                    "client_user_agent": client_user_agent,
                    "accept_encoding": accept_encoding,
                    "body": body_start}
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_data}")
            print(f"len headers: {len(headers_raw)}")
            print(f"len body: {content_length}")
            print(f"len received body: {len(body_start)}")
        
    
    def route_request(parsed_request: dict) -> dict:
        """Returns (status, body)"""
        request_target = parsed_request["request_target"]
        client_user_agent = parsed_request["client_user_agent"]
        request_method = parsed_request["request_method"]

        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            content_type = "text/plain"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            content_type = "text/plain"
            body = client_user_agent

        elif parts[1] == "files":
            filename = parts[2]
            directory = sys.argv[2]
            full_path = os.path.join(directory, filename)
            try:
                if request_method == "GET":
                    with open(full_path, mode="rb") as f: 
                        file_bytes = f.read()
                        status = "200 OK"
                        content_type = "application/octet-stream"
                        body = file_bytes
                elif request_method == "POST":
                    body = parsed_request["body"]
                    with open(full_path, mode="w") as f: 
                        file_bytes = f.write(body)
                        status = "201 Created"
                        content_type = "application/octet-stream"
                else:
                    status = "405 Method Not Allowed"
                    content_type = "text/plain"
                    body = ""

            except FileNotFoundError:
                status = "404 Not Found"
                content_type = "text/plain"
                body = ""                
                

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            content_type = "text/plain"
            body = parts[2]

        else:
            status = "404 Not Found"
            content_type = ""
            body = ""

        return {"status": status,
                "content_type": content_type,
                "body": body}
    
    def apply_compressed_response(response_encoding: list, route_result: dict):
        """Apply the compression to the response if needed."""
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]  

        # Compress the body with gzip package
        compressed_body = gzip.compress(bytes(body, 'utf-8'))

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Encoding: {', '.join(response_encoding)}",
            f"Content-Length: {len(compressed_body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(compressed_body, str):
            body_bytes = compressed_body.encode()
        else:
            body_bytes = compressed_body

        return header_bytes + body_bytes
    
    def create_response(route_result: dict):
        """
        Get the route result. Returns the response as bytes.
        """
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Length: {len(body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(body, str):
            body_bytes = body.encode()
        else:
            body_bytes = body

        return header_bytes + body_bytes
    
    def handle_request(client_socket):
        try:
            request_data = b""
            while b"\r\n\r\n" not in request_data:
                request_data += client_socket.recv(1024)

            parsed_request = parse_client_request(request_data, client_socket)

            route_result = route_request(parsed_request)

            accept_encoding = parsed_request["accept_encoding"]

            server_accepted_encoding = ["gzip"]
            response_encoding = []

            for e in accept_encoding:
                if e in server_accepted_encoding:
                    response_encoding.append(e)
                    break

            if response_encoding:
                server_response = apply_compressed_response(response_encoding, route_result)
            else:
                server_response  = create_response(route_result)

            client_socket.send(server_response)

        except Exception as e:
            print(f"An error occured: {e}")

        finally:
            client_socket.close()
      
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        while True:
            client_socket, address_info = server_socket.accept() 
            hostaddr = address_info[0]
            port = address_info[1]

            thread = threading.Thread(target=handle_request, args=(client_socket,))
            thread.start() 

    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    main()


## Extension: Persistent connections

## 12. Persistent connections

In this stage, you'll add support for persistent HTTP connections. By default, HTTP/1.1 uses persistent connections, allowing the same TCP connection to be reused for multiple requests.

**Tests**

The tester will run your server using:

```bash
$ ./your_program.sh
````

Then it will send two sequential HTTP requests over the same TCP connection:

```bash
$ curl --http1.1 -v http://localhost:4221/echo/banana --next http://localhost:4221/user-agent -H "User-Agent: blueberry/apple-blueberry"
```

**Expected Behavior**

* The server must keep the TCP connection open after the first request
* It must process both requests on the same connection
* It must return appropriate responses for each request

**Notes**

* In HTTP/1.1, connections are persistent unless the client includes the header `Connection: close`
* Each request must be handled independently, even when using the same connection
* The server must not close the connection after processing a request unless instructed to do so


In [ ]:
import os
import sys
import gzip
import socket  
import threading

def main():

    def parse_client_request(request_data: bytes, client_socket: socket) -> dict:
        """
        Parse the request from the client.
        "GET /echo/pineapple HTTP/1.1\r\nHost: localhost:4221\r\nAccept-Encoding: encoding-1, gzip, encoding-2\r\n\r\n"
        """
        headers_raw, body_start = request_data.decode().split("\r\n\r\n")
        header_lines = headers_raw.split("\r\n")

        try:
            # Request line
            request_line = header_lines[0]
            request_method = request_line.split(" ")[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = header_lines[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")
            accept_encoding = headers.get('accept-encoding', "")

            # If received accept-encoding
            if accept_encoding:
                # List format
                accept_encoding = accept_encoding.split(", ")

            if request_method == "POST":
                content_length = int(headers.get('content-length', ""))
                while len(body_start) < content_length:
                    body_start += client_socket.recv(content_length - len(body_start))

            return {"request_target": request_target,
                    "request_method": request_method,
                    "client_user_agent": client_user_agent,
                    "accept_encoding": accept_encoding,
                    "body": body_start}
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_data}")
            print(f"len headers: {len(headers_raw)}")
            print(f"len body: {content_length}")
            print(f"len received body: {len(body_start)}")
        
    
    def route_request(parsed_request: dict) -> dict:
        """Returns (status, body)"""
        request_target = parsed_request["request_target"]
        client_user_agent = parsed_request["client_user_agent"]
        request_method = parsed_request["request_method"]

        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            content_type = "text/plain"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            content_type = "text/plain"
            body = client_user_agent

        elif parts[1] == "files":
            filename = parts[2]
            directory = sys.argv[2]
            full_path = os.path.join(directory, filename)
            try:
                if request_method == "GET":
                    with open(full_path, mode="rb") as f: 
                        file_bytes = f.read()
                        status = "200 OK"
                        content_type = "application/octet-stream"
                        body = file_bytes
                elif request_method == "POST":
                    body = parsed_request["body"]
                    with open(full_path, mode="w") as f: 
                        file_bytes = f.write(body)
                        status = "201 Created"
                        content_type = "application/octet-stream"
                else:
                    status = "405 Method Not Allowed"
                    content_type = "text/plain"
                    body = ""

            except FileNotFoundError:
                status = "404 Not Found"
                content_type = "text/plain"
                body = ""                
                

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            content_type = "text/plain"
            body = parts[2]

        else:
            status = "404 Not Found"
            content_type = ""
            body = ""

        return {"status": status,
                "content_type": content_type,
                "body": body}
    
    def apply_compressed_response(response_encoding: list, route_result: dict):
        """Apply the compression to the response if needed."""
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]  

        # Compress the body with gzip package
        compressed_body = gzip.compress(bytes(body, 'utf-8'))

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Encoding: {', '.join(response_encoding)}",
            f"Content-Length: {len(compressed_body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(compressed_body, str):
            body_bytes = compressed_body.encode()
        else:
            body_bytes = compressed_body

        return header_bytes + body_bytes
    
    def create_response(route_result: dict):
        """
        Get the route result. Returns the response as bytes.
        """
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Length: {len(body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(body, str):
            body_bytes = body.encode()
        else:
            body_bytes = body

        return header_bytes + body_bytes
    
    def handle_request(client_socket):
        try:
            # Keep reading requests from the same client socket by using while
            # Previous behavior: Read → Respond → Exit
            # Required behavior: Read → Respond → Repeat → Until connection is closed 
            while True:
                request_data = b""
                while b"\r\n\r\n" not in request_data:
                    request_data += client_socket.recv(1024)

                parsed_request = parse_client_request(request_data, client_socket)

                route_result = route_request(parsed_request)

                accept_encoding = parsed_request["accept_encoding"]

                server_accepted_encoding = ["gzip"]
                response_encoding = []

                for e in accept_encoding:
                    if e in server_accepted_encoding:
                        response_encoding.append(e)
                        break

                if response_encoding:
                    server_response = apply_compressed_response(response_encoding, route_result)
                else:
                    server_response  = create_response(route_result)

                client_socket.send(server_response)

        except Exception as e:
            print(f"An error occured: {e}")

        finally:
            client_socket.close()
      
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        while True:
            client_socket, address_info = server_socket.accept() 
            hostaddr = address_info[0]
            port = address_info[1]

            thread = threading.Thread(target=handle_request, args=(client_socket,))
            thread.start() 

    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    main()


## 13. Concurrent persistent connections

In this stage, you'll extend your server to support multiple **concurrent** persistent connections.

**Tests**

The tester will run your server using:

```bash
$ ./your_program.sh
````

Then it will create two persistent TCP connections and send multiple requests over each:

```bash
$ curl --http1.1 -v http://localhost:4221/user-agent -H "User-Agent: orange/mango-grape" --next http://localhost:4221/echo/apple
```

**Expected Behavior**

* The server must handle multiple concurrent TCP connections
* It must keep each connection open for multiple requests
* Requests on each connection must be processed independently
* Appropriate responses must be returned for all requests

**Notes**

* Each connection should be handled in isolation
* The server must manage the state of each connection separately
* Connections can be processed concurrently without blocking each other


In [ ]:
import os
import sys
import gzip
import socket  
import threading

def main():

    def parse_client_request(request_data: bytes, client_socket: socket) -> dict:
        """
        Parse the request from the client.
        "GET /echo/pineapple HTTP/1.1\r\nHost: localhost:4221\r\nAccept-Encoding: encoding-1, gzip, encoding-2\r\n\r\n"
        """
        headers_raw, body_start = request_data.decode().split("\r\n\r\n")
        header_lines = headers_raw.split("\r\n")

        try:
            # Request line
            request_line = header_lines[0]
            request_method = request_line.split(" ")[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = header_lines[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")
            accept_encoding = headers.get('accept-encoding', "")

            # If received accept-encoding
            if accept_encoding:
                # List format
                accept_encoding = accept_encoding.split(", ")

            if request_method == "POST":
                content_length = int(headers.get('content-length', ""))
                while len(body_start) < content_length:
                    body_start += client_socket.recv(content_length - len(body_start))

            return {"request_target": request_target,
                    "request_method": request_method,
                    "client_user_agent": client_user_agent,
                    "accept_encoding": accept_encoding,
                    "body": body_start}
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_data}")
            print(f"len headers: {len(headers_raw)}")
            print(f"len body: {content_length}")
            print(f"len received body: {len(body_start)}")
        
    
    def route_request(parsed_request: dict) -> dict:
        """Returns (status, body)"""
        request_target = parsed_request["request_target"]
        client_user_agent = parsed_request["client_user_agent"]
        request_method = parsed_request["request_method"]

        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            content_type = "text/plain"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            content_type = "text/plain"
            body = client_user_agent

        elif parts[1] == "files":
            filename = parts[2]
            directory = sys.argv[2]
            full_path = os.path.join(directory, filename)
            try:
                if request_method == "GET":
                    with open(full_path, mode="rb") as f: 
                        file_bytes = f.read()
                        status = "200 OK"
                        content_type = "application/octet-stream"
                        body = file_bytes
                elif request_method == "POST":
                    body = parsed_request["body"]
                    with open(full_path, mode="w") as f: 
                        file_bytes = f.write(body)
                        status = "201 Created"
                        content_type = "application/octet-stream"
                else:
                    status = "405 Method Not Allowed"
                    content_type = "text/plain"
                    body = ""

            except FileNotFoundError:
                status = "404 Not Found"
                content_type = "text/plain"
                body = ""                
                

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            content_type = "text/plain"
            body = parts[2]

        else:
            status = "404 Not Found"
            content_type = ""
            body = ""

        return {"status": status,
                "content_type": content_type,
                "body": body}
    
    def apply_compressed_response(response_encoding: list, route_result: dict):
        """Apply the compression to the response if needed."""
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]  

        # Compress the body with gzip package
        compressed_body = gzip.compress(bytes(body, 'utf-8'))

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Encoding: {', '.join(response_encoding)}",
            f"Content-Length: {len(compressed_body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(compressed_body, str):
            body_bytes = compressed_body.encode()
        else:
            body_bytes = compressed_body

        return header_bytes + body_bytes
    
    def create_response(route_result: dict):
        """
        Get the route result. Returns the response as bytes.
        """
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Length: {len(body)}",
        ]            
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(body, str):
            body_bytes = body.encode()
        else:
            body_bytes = body

        return header_bytes + body_bytes
    
    def handle_request(client_socket):
        try:
            while True:
                request_data = b""
                while b"\r\n\r\n" not in request_data:
                    request_data += client_socket.recv(1024)

                parsed_request = parse_client_request(request_data, client_socket)

                route_result = route_request(parsed_request)

                accept_encoding = parsed_request["accept_encoding"]

                server_accepted_encoding = ["gzip"]
                response_encoding = []

                for e in accept_encoding:
                    if e in server_accepted_encoding:
                        response_encoding.append(e)
                        break

                if response_encoding:
                    server_response = apply_compressed_response(response_encoding, route_result)
                else:
                    server_response  = create_response(route_result)

                client_socket.send(server_response)

        except Exception as e:
            print(f"An error occured: {e}")

        finally:
            client_socket.close()
      
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        while True:
            client_socket, address_info = server_socket.accept() 
            hostaddr = address_info[0]
            port = address_info[1]

            # Handling concurrent persistent connections
            thread = threading.Thread(target=handle_request, args=(client_socket,))
            thread.start() 

    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    main()


## 14. Connection closure

In this stage, you'll add support for explicitly closing a connection using the `Connection: close` header.

**Tests**

The tester will run your server using:

```bash
$ ./your_program.sh
````

Then it will send two sequential requests:

1. A regular request (connection should remain open)
2. A request with the `Connection: close` header (connection should be closed)

```bash
$ curl --http1.1 -v http://localhost:4221/echo/orange --next http://localhost:4221/ -H "Connection: close"
```

**Expected Behavior**

* The server must keep the TCP connection open after the first request
* The server must close the connection after the second request if it includes the `Connection: close` header
* The response to the second request must include `Connection: close`

**Notes**

* The `Connection: close` header indicates the client wants to close the connection after the current request
* The server should mirror the header in its response
* The TCP connection should be closed immediately after sending the response



In [ ]:
import os
import sys
import gzip
import socket  
import threading

def main():

    def parse_client_request(request_data: bytes, client_socket: socket) -> dict:
        """
        Parse the request from the client.
        "GET /echo/pineapple HTTP/1.1\r\nHost: localhost:4221\r\nAccept-Encoding: encoding-1, gzip, encoding-2\r\n\r\n"
        """
        headers_raw, body_start = request_data.decode().split("\r\n\r\n")
        header_lines = headers_raw.split("\r\n")

        try:
            # Request line
            request_line = header_lines[0]
            request_method = request_line.split(" ")[0]
            request_target = request_line.split(" ")[1]

            # Header
            headers = {}
            header_part = header_lines[1:]
            for line in header_part:
                if ":" in line:
                    key, value = line.split(": ")
                    headers[key.lower()]= value

            client_user_agent = headers.get('user-agent', "")
            accept_encoding = headers.get('accept-encoding', "")

            # If received accept-encoding
            if accept_encoding:
                # List format
                accept_encoding = accept_encoding.split(", ")

            if request_method == "POST":
                content_length = int(headers.get('content-length', ""))
                while len(body_start) < content_length:
                    body_start += client_socket.recv(content_length - len(body_start))

            return {"request_target": request_target,
                    "request_method": request_method,
                    "client_user_agent": client_user_agent,
                    "accept_encoding": accept_encoding,
                    "connection": headers.get("connection", "").lower(),
                    "body": body_start}
        
        except Exception as e:
            print(f"An error occured: {e}")
            print("Debug")
            print(f"Request Data: {request_data}")
            print(f"len headers: {len(headers_raw)}")
            print(f"len body: {content_length}")
            print(f"len received body: {len(body_start)}")
        
    
    def route_request(parsed_request: dict) -> dict:
        """Returns (status, body)"""
        request_target = parsed_request["request_target"]
        client_user_agent = parsed_request["client_user_agent"]
        request_method = parsed_request["request_method"]

        parts = request_target.split("/")
        if request_target == "/":
            status = "200 OK"
            content_type = "text/plain"
            body = ""       

        elif parts[1] == "user-agent":
            status = "200 OK"
            content_type = "text/plain"
            body = client_user_agent

        elif parts[1] == "files":
            filename = parts[2]
            directory = sys.argv[2]
            full_path = os.path.join(directory, filename)
            try:
                if request_method == "GET":
                    with open(full_path, mode="rb") as f: 
                        file_bytes = f.read()
                        status = "200 OK"
                        content_type = "application/octet-stream"
                        body = file_bytes
                elif request_method == "POST":
                    body = parsed_request["body"]
                    with open(full_path, mode="w") as f: 
                        file_bytes = f.write(body)
                        status = "201 Created"
                        content_type = "application/octet-stream"
                else:
                    status = "405 Method Not Allowed"
                    content_type = "text/plain"
                    body = ""

            except FileNotFoundError:
                status = "404 Not Found"
                content_type = "text/plain"
                body = ""                
                

        elif len(parts) == 3 and parts[1] == "echo":
            status = "200 OK"
            content_type = "text/plain"
            body = parts[2]

        else:
            status = "404 Not Found"
            content_type = ""
            body = ""

        return {"status": status,
                "content_type": content_type,
                "body": body}
    
    def apply_compressed_response(response_encoding: list, route_result: dict, connection_close: bool = False):
        """Apply the compression to the response if needed."""
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]  

        compressed_body = gzip.compress(bytes(body, 'utf-8'))

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Encoding: {', '.join(response_encoding)}",
            f"Content-Length: {len(compressed_body)}",
        ]            
        if connection_close:
            headers.append("Connection: close")
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(compressed_body, str):
            body_bytes = compressed_body.encode()
        else:
            body_bytes = compressed_body

        return header_bytes + body_bytes
    
    def create_response(route_result: dict, connection_close: bool = False):
        """
        Get the route result. Returns the response as bytes.
        """
        status = route_result["status"]
        content_type = route_result["content_type"]
        body = route_result["body"]

        headers = [
            f"HTTP/1.1 {status}",
            f"Content-Type: {content_type }",
            f"Content-Length: {len(body)}",
        ]            

        if connection_close:
            headers.append("Connection: close")
        header_str = "\r\n".join(headers) + "\r\n\r\n"

        header_bytes = header_str.encode()

        if isinstance(body, str):
            body_bytes = body.encode()
        else:
            body_bytes = body

        return header_bytes + body_bytes
    
    def handle_request(client_socket):
        try:
            while True:
                connection_close = False 
                request_data = b""
                while b"\r\n\r\n" not in request_data:
                    request_data += client_socket.recv(1024)

                parsed_request = parse_client_request(request_data, client_socket)

                if parsed_request["connection"] == "close":
                    # Check if the connection close in the header
                    connection_close = True

                route_result = route_request(parsed_request)

                accept_encoding = parsed_request["accept_encoding"]

                server_accepted_encoding = ["gzip"]
                response_encoding = []

                for e in accept_encoding:
                    if e in server_accepted_encoding:
                        response_encoding.append(e)
                        break

                if response_encoding:
                    server_response = apply_compressed_response(response_encoding, route_result, connection_close)
                else:
                    server_response  = create_response(route_result, connection_close)

                client_socket.send(server_response)

                if connection_close:
                    break  # If connection is closed flag, break the while loop

        except Exception as e:
            print(f"An error occured: {e}")

        finally:
            client_socket.close()
      
    try:
        server_socket = socket.create_server(("localhost", 4221), reuse_port=True)
        print("Listening to http://localhost:4221")

        while True:
            client_socket, address_info = server_socket.accept() 
            hostaddr = address_info[0]
            port = address_info[1]

            thread = threading.Thread(target=handle_request, args=(client_socket,))
            thread.start() 

    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    main()


# End!

**Future Improvement**
1. Cleaner Design Alternative: Use a Response object (dict or class): 
- Instead of passing lots of individual parameters, pass a single structured response object that contains all the parameters.

